In [ ]:
import pandas as pd
import re
import time
import requests
from bs4 import BeautifulSoup
import csv

In [ ]:
df = pd.read_csv(r'C:\Users\malin\Downloads\games.csv')

In [ ]:
df.drop(index=df.index[0], axis=0,inplace=True)
df

,0
1,https://www.metacritic.com/game/last-train-home/
2,https://www.metacritic.com/game/call-of-duty-m...
3,https://www.metacritic.com/game/super-mario-rpg/
4,https://www.metacritic.com/game/alan-wake-ii/
5,https://www.metacritic.com/game/steamworld-build/
...,...
12439,https://www.metacritic.com/game/operencia-the-...
12440,https://www.metacritic.com/game/empire-earth-iii/
12441,https://www.metacritic.com/game/frozen-synapse...
12442,https://www.metacritic.com/game/pirates-of-the...


In [ ]:
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"}
html_text = requests.get("https://www.metacritic.com/browse/game/?releaseYearMin=1910&releaseYearMax=2023", headers=headers)

In [ ]:
url_list = df.values.tolist()

In [ ]:
games_dict = {'Title':[], 'Release_date':[], 'Metascore':[], 'User_score':[]}

url_list = df['0'].tolist()

for url in url_list:
    retries = 0
    while retries < 3:
        try:
            print(f"Processing URL: {url}")
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')

                # Extract title
                title_element = soup.find('div', class_=re.compile('^c-productHero_title'))
                title = title_element.get_text() if title_element else None
                if title:
                    games_dict['Title'].append(title)

                # Extract release date
                release_container = soup.find('div', class_="c-productHero_score-container")
                if release_container:
                    release_span = release_container.find('div', class_="g-text-xsmall")
                    if release_span:
                        release_date_element = release_span.find('span', class_="u-text-uppercase")
                        release_date = release_span.get_text(strip=True) if release_date_element else None
                        games_dict['Release_date'].append(release_date)
                    else:
                        games_dict['Release_date'].append(None)

                else:
                    games_dict['Release_date'].append(None)

                # Extract scores
                scores = soup.find_all('div', class_='c-productScoreInfo_scoreNumber')
                meta = scores[0].find('span').text if scores and len(scores) > 0 else None
                user = scores[1].find('span').text if scores and len(scores) > 1 else None

                games_dict['Metascore'].append(meta if meta else None)
                games_dict['User_score'].append(user if user else None)

                time.sleep(2)
                break

            elif response.status_code == 500:
                retries += 1
                print(f"Retrying {url} - Retry attempt: {retries}")
                time.sleep(2)
                continue  #Got a lot of 500 errors on the first attempt running the code, added a loop to retry on status error 500

            else:
                print(f"Failed to get data from {url}. Status code: {response.status_code}")
                break

        except requests.RequestException as e:
            print(f"Error accessing URL: {url}. Error: {e}")
            break

print(games_dict)

Processing URL: https://www.metacritic.com/game/last-train-home/
Processing URL: https://www.metacritic.com/game/call-of-duty-modern-warfare-iii/
Processing URL: https://www.metacritic.com/game/super-mario-rpg/
Processing URL: https://www.metacritic.com/game/alan-wake-ii/
Processing URL: https://www.metacritic.com/game/steamworld-build/
Processing URL: https://www.metacritic.com/game/persona-5-tactica/
Processing URL: https://www.metacritic.com/game/like-a-dragon-gaiden-the-man-who-erased-his-name/
Processing URL: https://www.metacritic.com/game/marvels-spider-man-2/
Processing URL: https://www.metacritic.com/game/the-legend-of-zelda-ocarina-of-time/
Processing URL: https://www.metacritic.com/game/grand-theft-auto-iv/
Processing URL: https://www.metacritic.com/game/tony-hawks-pro-skater-2/
Processing URL: https://www.metacritic.com/game/soulcalibur/
Processing URL: https://www.metacritic.com/game/super-mario-galaxy/
Processing URL: https://www.metacritic.com/game/grand-theft-auto-v/
Pr

In [ ]:
for url in url_list:
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')

  platforms = []
  platforms_div = soup.find('div', class_='c-PageProductGame_row')
  if platforms_div:
    all_platforms = platforms_div.find('div', class_='c-gamePlatformsSection_list')
    if all_platforms:
      platform_elements = all_platforms.find_all('svg', class_='c-gamePlatformLogo_icon')
      for element in platform_elements:
        title_tag = element.find('title')
        if title_tag:
          platform_name = title_tag.getText(strip=True)
          platforms.append(platform_name)
  platforms_combined = ', '.join(platforms)
  games_dict['Platforms'] = platforms_combined

In [ ]:

def cleaning_dict(games_dict):
    games_dict_cleaned = {}
    for key, value in games_dict.items():
            cleaned_list = [item.strip() if isinstance(item, str) else item for item in value]
            cleaned_list = [re.sub(r'Released On:', '', item) if isinstance(item, str) else item for item in cleaned_list]
            games_dict_cleaned[key] = cleaned_list
    return games_dict_cleaned

#games_dict_cleaned.clear()
games_dict_cleaned = cleaning_dict(games_dict)
print(games_dict_cleaned)

{'Title': ['Last Train Home', 'Call of Duty: Modern Warfare III', 'Super Mario RPG', 'Alan Wake II', 'SteamWorld Build', 'Persona 5 Tactica', 'Like a Dragon Gaiden: The Man Who Erased His Name', "Marvel's Spider-Man 2", 'The Legend of Zelda: Ocarina of Time', 'Grand Theft Auto IV', "Tony Hawk's Pro Skater 2", 'SoulCalibur', 'Super Mario Galaxy', 'Grand Theft Auto V', 'Halo: Combat Evolved', 'NFL 2K1', 'Grand Theft Auto III', 'Super Mario Galaxy 2', 'Metroid Prime', "Tony Hawk's Pro Skater 3", 'The Legend of Zelda: Breath of the Wild', 'Super Mario Odyssey', 'Red Dead Redemption 2', 'Disco Elysium: The Final Cut', 'Perfect Dark (2000)', 'The Legend of Zelda: Twilight Princess', 'The Orange Box', 'Half-Life 2', 'BioShock', 'GoldenEye 007', 'Gran Turismo', 'Metal Gear Solid 2: Sons of Liberty', 'Uncharted 2: Among Thieves', 'Tekken 3', 'Half-Life', 'Grand Theft Auto Double Pack', 'Mass Effect 2', "Baldur's Gate 3", 'The House in Fata Morgana - Dreams of the Revenants Edition -', 'Elden Ri

In [ ]:
df = pd.DataFrame.from_dict(games_dict_cleaned, orient='index')
#df.drop(df.index, inplace=True)

In [ ]:
pivoted = df.pivot(index="title", columns="variable")

In [ ]:
df.to_csv(r"C:\Users\malin\Downloads\metacritic_games.csv",index=False)